In [0]:



from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
start = '2019-01-01'
end = '2019-12-31'
granularity = 'quarterly'
category_daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-load.v3/fact/").where("granularity='daily' and date between '{}' and '{}'".format(start, end))
agg_df = category_daily_df.groupBy('app_id', 'country_code', 'device_code', 'category_id').agg(
    sum('est_free_app_download').alias('free_app_download'), sum('est_paid_app_download').alias('paid_app_download'), 
        sum('est_revenue').alias('revenue'))
agg_df.createOrReplaceTempView("agg_df")
if granularity == 'weekly' or granularity == 'monthly':
    pre_agg_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category.v3/fact/").where("granularity='{}' and date='{}' and data_stage='final'".format(granularity, end))
else:
    pre_agg_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-pre-aggr.v3/fact/").where("granularity='{}' and date='{}' and data_stage='final'".format(granularity, end))
pre_agg_df.createOrReplaceTempView("pre_agg_df")
diff_df1 = spark.sql("""select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from agg_df 
                        except all 
                        select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from pre_agg_df""")
diff_df2 = spark.sql("""select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from pre_agg_df 
                        except all 
                        select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from agg_df""")
print agg_df.count(), pre_agg_df.count()
print diff_df1.count(), diff_df2.count()
diff_df1.show()
diff_df2.show()


In [0]:


start = '2020-01-01'
end = '2020-03-31'
granularity = 'quarterly'
quarterly_category_bucket ="store.app-est-category-pre-aggr.v3"
quarterly_est_bucket= "store.app-est-pre-aggr.v3"



category_daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
    "{}/fact/".format(quarterly_category_bucket)).where("granularity='{}' and date between '{}' and '{}'".format(granularity, start, end))
category_daily_df.createOrReplaceTempView("category_daily_df")
spark.sql(""" select date, app_id, country_code, device_code, category_id,
                    free_app_download,paid_app_download, revenue
                from  category_daily_df""").createOrReplaceTempView("unpivot_category_daily_df")
# spark.sql("""select app_id, country_code, device_code, category_id, metric, sum(est) as est
#                 from unpivot_category_daily_df
#                 where est is not null
#                 group by
#                 app_id,
#                 country_code,
#                 device_code,
#                 category_id,
#                 metric
# """).createOrReplaceTempView("unpivot_category_agg_df")
est_daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
    "{}/fact/".format(quarterly_est_bucket)).where("granularity='{}' and date between '{}' and '{}'".format(granularity, start, end))
est_daily_df.createOrReplaceTempView("est_daily_df")
spark.sql(""" select app_id, country_code, device_code, 
                 free_app_download, paid_app_download, revenue
            FROM est_daily_df """).createOrReplaceTempView("unpivot_est_agg_df")
spark.sql("""select d1.app_id, d1.country_code, d1.device_code, d1.category_id, d2.free_app_download, d2.paid_app_download, d2.revenue
                from unpivot_category_daily_df as d1 
                join unpivot_est_agg_df as d2 
                on
                d1.app_id=d2.app_id
                and d1.country_code=d2.country_code
                and d1.device_code=d2.device_code
                and d1.free_app_download=d2.free_app_download
                and d1.paid_app_download=d2.paid_app_download
                and d1.revenue=d2.revenue """).createOrReplaceTempView("unpivot_agg_df")
# spark.sql(""" select * FROM unpivot_agg_df
#                 pivot(
#                     max(est) FOR metric IN ('free_app_download','revenue', 'paid_app_download')
#                 )
# """).createOrReplaceTempView("agg_df")
spark.sql("""select * from unpivot_agg_df where category_id=100009 and country_code='WW' and device_code='ios-phone' order by paid_app_download desc, app_id desc limit 1000""").show(1000)




In [0]:

start = '2019-12-29'
end = '2020-01-04'

daily_category_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-load.v3/fact/").where("granularity='daily' and date between '{}' and '{}' and app_id=1225683141 and country_code='US' and device_code='ios-phone'".format(start, end)).show(200)

weekly_df_category = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category.v3/fact/").where("granularity='weekly' and date between '{}' and '{}'  and app_id=1225683141 and country_code='US' and device_code='ios-phone'".format(start, end))
weekly_df_category.createOrReplaceTempView("weekly_category")

weekly_df_est = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v3/fact/").where("granularity='weekly' and date between '{}' and '{}'  and app_id=1225683141 and country_code='US' and device_code='ios-phone'".format(start, end))
weekly_df_est.createOrReplaceTempView("weekly_est")

spark.sql("")






In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-pre-aggr.v3/fact/granularity=quarterly/ --recursive | sort -n
 
 

In [0]:

start = '2019-12-29'
end = '2020-01-04'
daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-load.v3/fact/").where("granularity='daily' and date between '{}' and '{}'".format(start, end))
daily_df.createOrReplaceTempView("daily_df")
quarterly_df_ho = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v3/fact/").where("granularity='weekly' and date='{}' and data_stage='final'".format(end))
quarterly_df_ho.createOrReplaceTempView("quarterly_df")
spark.sql("select date, app_id,device_code,country_code,category_id,free_app_download from quarterly_df where app_id=1009836739 and country_code='AM' and device_code='ios-phone'").show()
spark.sql("select date, app_id,device_code,country_code,category_id,est_free_app_download from daily_df where app_id=1009836739 and country_code='AM' and device_code='ios-phone' order by category_id").show()

In [0]:

start = '2019-09-29'
end = '2019-10-05'
daily_df_1 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-load.v3/fact/").where("granularity='daily' and date between '{}' and '{}'".format(start, end))
daily_df_1.where("app_id=538212549 and country_code='US' and device_code='ios-phone'").show(999)




In [0]:

start = '2019-10-05'
end = '2019-10-05'
daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category.v3/fact/").where("granularity='weekly' and date between '{}' and '{}'".format(start, end))
daily_df.filter("app_id=538212549 and country_code='US' and device_code='ios-phone'").show(999)


# spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category.v3/fact").where("granularity='monthly' and date between '{}' and '{}' and app_id=538212549 and country_code='MU' and device_code='ios-phone' and category_id=100000".format(start, end)).show()

In [0]:

import datetime
start = "2016-08-28"
end = "2020-03-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append(str(real_date1 + datetime.timedelta(days)))

for x in sar_list:
    print x





In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F


date='2020-02-23'

def plproxy_row(date):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select app_id from plproxy.execute_select_nestloop($$ 
                select app_id from (
                    select distinct app_id,store_id ,device_id, date, est_organic_download_share
                        from da.app_da_daily_estimate_1001 where date='{}' and store_id=10 )
                        as prod $$) tbl 
              (app_id bigint ) order by app_id  ;
            '''.format(
            date
    )

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(app_id=r[0]) for r in rows]


def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select app_id from store.store_est_fact_v6 where date = '{}' and country_code='US' and est_paid_download !=0  and device_code='android-all' order by app_id".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result

    result = get_data_in_citus(date)
    return [Row(app_id=r[0]) for r in result]


plproxy_result = plproxy_row(date)
citus_result = citus_row(date)

df_plproxy = spark.createDataFrame(plproxy_result).cache()
df_plproxy.createOrReplaceTempView("temp_plproxy")

citus_df = spark.createDataFrame(citus_result).cache()
citus_df.createOrReplaceTempView("temp_citus")
spark.sql("select * from temp_citus except select * from temp_plproxy ").show()
diff_df_list = spark.sql("select * from temp_plproxy except select * from temp_citus ").collect()
print 'not equal count', len(diff_df_list)
diff_list = [str(x[0]) for x in diff_df_list]
df = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date={}".format(date)).cache()
df.createOrReplaceTempView("temp")
spark.sql("select * from temp where id in ({}) and store_id=10 and feed in (0,1)".format(",".join(diff_list))).show()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
select distinct app_id,store_id,device_id, date from plproxy.execute_select_nestloop(\$proxy\$ 
select  distinct app_id,store_id ,device_id, date
    from da.app_da_daily_estimate_2001 where date='2019-12-22' and app_id = 1225683141 and store_id=143441
     limit 5 \$proxy\$) tbl 
      (app_id BIGINT, store_id INT, device_id SMALLINT, date Date )  ;

EOF


# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select sum(cnt) as sum from plproxy.execute_select_nestloop(\$proxy\$ 
# select count(1) as cnt from (
# select distinct app_id,store_id ,device_id, date, est_organic_download_share
#     from da.app_da_daily_estimate_1001 where ( date  between '2020-01-01' and '2020-01-01') and store_id=5 )
#     as prod
#  \$proxy\$) tbl 
#       (cnt bigint )  ;

# EOF

# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select app_id from plproxy.execute_select_nestloop(\$proxy\$ 
# select app_id from (
# select distinct app_id,store_id ,device_id, date, est_organic_download_share
#     from da.app_da_daily_estimate_1001 where ( date  between '2020-01-01' and '2020-01-01') and store_id=5 )
#     as prod
#  \$proxy\$) tbl 
#       (app_id bigint ) order by app_id  ;

# EOF



In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- est_free_app_download + est_paid_app_download = organic_download + paid_download

--select * from store_est_fact_v6 where date = '2020-03-05' and country_code='AU' and device_code='ios-phone' and est_paid_download is not null order by est_paid_download desc limit 5
-- select * from store_est_category_fact_v7 where date = '2020-01-01' and country_code='US'  and device_code='android-all' and app_id = 20600006409047;

-- select (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_free_paid, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid from store_est_fact_v6 where date between '2019-08-29' and '2020-05-30' limit 5 ;


-- select * from store_est_category_fact_v7 where app_id=446366839 and country_code='AU'  and device_code='ios-phone' and date between '2017-06-05' and '2017-06-05' limit 5 ;


select (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_total, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid  from store_est_fact_v1 where  country_code='US' and est_paid_download !=0  and device_code='ios-phone' and date between '2020-05-10' and '2020-05-30' limit 3 ;
select  (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_total, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid from store_est_category_fact_v1 where  country_code='US' and est_paid_download !=0  and device_code='ios-phone' and date between '2020-05-10' and '2020-05-30' limit 3 ;

select * from store_est_fact_v1 where date between '2020-05-15' and '2020-05-15' and country_code='US' and device_code='ios-phone' limit 5 

EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- select * from store_est_t_w_fact_v6 where device_code='ios-tablet' and country_code='GB' and date ='2019-06-15' and est_paid_download!=0 limit 3;
-- select * from store_est_t_m_fact_v6 where device_code='android-all' and country_code='JP' and date ='2019-07-31'  and est_paid_download!=0  limit 3;
--select * from store_est_t_y_fact_v6 where device_code='ios-phone' and country_code='WW' and date ='2019-12-31' and est_paid_download!=0 limit 3;
--select * from store_est_t_y_fact_v6 where device_code='android-all' and country_code='WW' and date ='2019-12-31' and est_paid_download!=0 limit 3;

-- select * from store_est_category_t_q_fact_v7 limit 3;
-- select * from store_est_category_t_y_fact_v7 limit 3;

--select count(distinct date) from store_est_category_t_w_fact_v7 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30'  ;
--select distinct date from store_est_t_m_fact_v6 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date 
-- select distinct date from store_est_category_t_m_fact_v7 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date;


-- select * from store_est_category_fact_v7 where date = '2020-01-01' and country_code='US'  and device_code='android-all' and app_id = 20600006409047;

-- select (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_free_paid, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid from store_est_fact_v6 where date between '2019-08-29' and '2020-05-30' limit 5 ;


-- select * from store_est_category_fact_v7 where app_id=446366839 and country_code='AU'  and device_code='ios-phone' and date between '2017-06-05' and '2017-06-05' limit 5 ;



-- select sum(est_organic_download), sum(est_paid_download) from store_est_fact_v6 where date between '2019-10-01' and '2019-10-31' and device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;
-- select * from store_est_t_m_fact_v6 where device_code='ios-phone' and country_code='US' and date ='2019-10-31' and app_id=1225683141 limit 3;
--select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2019-10-01' and '2019-12-31' and --device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;
--select * from store_est_t_q_fact_v6 where device_code='ios-phone' and country_code='US' and date ='2019-12-31' and app_id=1225683141 limit 3;




--select distinct date from store_est_t_q_fact_v6 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date;
--select distinct date from store_est_t_y_fact_v6 where country_code='US' and est_paid_download !=0 and est_paid_download is not null and device_code='ios-phone' and date between '2016-08-26' and '2020-05-30' order by date



--select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2019-10-01' and '2019-10-31' and device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;
-- select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2019-11-01' and '2019-11-30' and device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;



select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2015-10-01' and '2015-12-31' and device_code='ios-phone' and country_code='US' limit 3;


select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_t_q_fact_v6 where date between '2015-12-31' and '2015-12-31' and device_code='ios-phone' and country_code='US'  limit 3;


# select date, sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_t_w_fact_v6 where date between '2019-10-01' and '2019-12-31' and device_code='ios-phone' and country_code='US' and app_id=1225683141 group by date limit 3;


# select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2019-12-22' and '2019-12-28' and device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;


# select date, sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_t_w_fact_v6 where date between '2019-12-22' and '2019-12-28' and device_code='ios-phone' and country_code='US' and app_id=1225683141 group by date limit 3;



# select date, sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_t_m_fact_v6 where date between '2019-10-01' and '2019-12-31' and device_code='ios-phone' and country_code='US' and app_id=1225683141 group by date limit 3;


# select sum(est_free_app_download) as sum_download, sum(est_revenue) as sum_revenue, sum(est_organic_download) as sum_organic_download, sum(est_paid_download) as sum_paid_download from store_est_fact_v6 where date between '2019-10-01' and '2019-12-31' and device_code='ios-phone' and country_code='US' and app_id=1225683141 limit 3;






EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

-- est_free_app_download + est_paid_app_download = organic_download + paid_download
-- select * from store_est_fact_v6 where date = '2016-08-26' limit 5 ;
-- select (sum(est_free_app_download) + sum(est_paid_app_download)) as sum_free_paid, (sum(est_paid_download) + sum(est_organic_download)) as sum_orgainc_paid from store_est_fact_v6 where date between '2019-08-29' and '2020-05-30' limit 5 ;


-- paid_download is Zero
-- select sum(est_paid_download) from store_est_category_fact_v7 where date < '2016-08-26' ;
-- select sum(est_paid_download) from store_est_fact_v6 where date < '2016-08-26' ;

-- three situations of download attribution with null value
select * from store_est_fact_v6 where date between '2016-08-29' and '2020-05-30' and est_paid_download is not null and est_paid_app_download is null limit 2 ;
select * from store_est_fact_v6 where date between '2016-08-29' and '2020-05-30' and est_paid_download is not null and est_paid_app_download is null and est_paid_download !=0  limit 2 ;
select * from store_est_fact_v6 where date between '2016-08-29' and '2020-05-30' and est_paid_download is null and est_paid_app_download is not null limit 2 ;
select * from store_est_fact_v6 where date between '2016-08-29' and '2020-05-30' and est_paid_download is null and est_paid_app_download is null limit 2 ;



EOF

In [0]:

import datetime

start = "2020-03-29"
end = "2020-05-02"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
print sar_list
for d in sar_list:
    daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where("granularity='daily' and date in ('%s')" % ("','".join(sar_list[0][1])))
    daily_download_attr.createOrReplaceTempView("daily_pre_load_data")
    
    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly' and date in ('%s')" % (sar_list[0][0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    

    spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

    spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()



In [0]:

import datetime

start = "2020-02-01"
end = "2020-04-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
# sar_list=list()
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))
#     if (real_date1 + datetime.timedelta(days)).weekday() == 5:
#         temp=list()
#         while dates:
#             temp.append(str(dates.pop()))
#         sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
# print sar_list

print dates
print dates[-1]
# sql_where = "granularity='daily' and date in ('%s')" % ("','".join(dates))
sql_where = "granularity='daily' and date between '2019-03-01' and '2019-03-31'"

print sql_where
daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where(sql_where)
daily_download_attr.createOrReplaceTempView("daily_pre_load_data")

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='monthly' and date='2019-03-31' ")
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    
spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()


spark.sql("select * from download_attribution_weekly_unified").show()

In [0]:

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='monthly'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")

test_date_list = spark.sql("select distinct date from download_attribution_weekly_unified").collect()

l = [str(d[0]) for d in test_date_list]
l.sort(reverse=False)
for x in l:
    print x

In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2020-05

In [0]:


daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where("granularity='daily' and date='2017-06-05'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_daily_unified")

spark.sql("select * from download_attribution_daily_unified").show()


In [0]:

import datetime

start = "2016-08-28"
end = "2020-05-31"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days))))

print sar_list

In [0]:

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")

test_date_list = spark.sql("select distinct date from download_attribution_weekly_unified").collect()

l = [str(d[0]) for d in test_date_list]
l.sort(reverse=False)
print l

In [0]:

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")

spark.sql("select * from download_attribution_weekly_unified where date = '2020-04-09' and granularity='weekly'  ").show()

In [0]:

import datetime
import calendar

start = "2010-07-04"
end = "2010-08-04"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()

month_day=list()
for days in xrange(date_range.days):
    month_day.append(real_date1 + datetime.timedelta(days))


test_list= sorted(list(set([ d.strftime("%Y-%m-%d")[:7] for d in month_day ])))

print test_list

# for m in test_list:
#     print calendar.monthrange(int(m[:4]),int(m[5:7]))
#     print "-".join([str(m[:4]), str(calendar.monthrange(int(m[:4]),int(m[-2:]))[0]), str(calendar.monthrange(int(m[:4]),int(m[-2:]))[1])])


def last_day_of_month(date):
    next_month = date.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)
print last_day_of_month(start)

In [0]:

import datetime

start = "2020-01-05"
end = "2020-02-02"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
print sar_list
for d in sar_list:
    daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where("granularity='daily' and date in ('%s')" % ("','".join(sar_list[0][1])))
    daily_download_attr.createOrReplaceTempView("daily_pre_load_data")
    
    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly' and date in ('%s')" % (sar_list[0][0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    

    spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

    spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()

